# Equation of state (EOS)

For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.eos import eos_workgraph

node = eos_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.eos import eos_workgraph
from aiida import load_profile

load_profile()

wg = eos_workgraph()
wg.to_html()

## Example: Silicon

### Prepare the inputs and submit the workflow


In [3]:
from ase.build import bulk
from ase import Atoms
from aiida import load_profile
from workgraph_collections.ase.espresso.eos import eos_workgraph

load_profile()

# create input structure node
atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = eos_workgraph(atoms=atoms,
                   computer="localhost",
                   scales=[0.95, 0.98, 1.0, 1.02, 1.05],
                   command="mpirun -np 2 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kpts=(4, 4, 4),
                   metadata=metadata)
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wg.nodes["fit_eos"].outputs["result"].value.get_dict()))



WorkGraph node created, PK: 55877

Result: 
B: 88.541865607602
v0: 41.224378448351
e0: -277.89647083333
v0: 41.224378448351
